In [193]:
import tool
import scipy
import pandas as pd
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from ipywidgets import interact, SelectMultiple, fixed
import warnings
warnings.filterwarnings('ignore')

In [194]:
"""
Data initialisation
"""

asset_index = pd.read_excel("data/asset_index.xlsx")

num_limit = np.random.randint(*(5,10))
num_limit = 5
index_list = asset_index['INDEX_CODE'].drop_duplicates().sample(num_limit).tolist()

asset_index['TRADE_DT'] = pd.to_datetime(asset_index['TRADE_DT'], format='%Y%m%d')
asset_index.sort_values(by='TRADE_DT', inplace=True)
asset_index.set_index('TRADE_DT', inplace=True)
asset_index = asset_index.pivot(columns='INDEX_CODE', values='CLOSE').ffill()[index_list].dropna()

print(index_list)
print(asset_index)

['h00300.CSI', 'GC.CMX', 'DOLLAR', 'CBA02001.CS', 'HSI.HI']
INDEX_CODE   h00300.CSI       GC.CMX  DOLLAR  CBA02001.CS        HSI.HI
TRADE_DT                                                               
2006-11-17  1591.526978   622.500000  7.8745   100.000000  19182.710938
2006-11-20  1623.187012   622.099976  7.8690    99.829002  18954.630859
2006-11-21  1642.637939   628.700012  7.8695    99.817001  19008.300781
2006-11-22  1654.639038   629.000000  7.8655    99.752998  19250.789062
2006-11-23  1665.730957   629.000000  7.8596    99.635002  19265.320312
...                 ...          ...     ...          ...           ...
2023-10-12  5138.992676  1881.599976  7.2941   227.643295  18238.210938
2023-10-13  5084.932129  1945.900024  7.3040   227.688995  17813.449219
2023-10-16  5034.107910  1932.900024  7.3040   227.787292  17640.359375
2023-10-17  5051.882324  1935.699951  7.3137   227.800598  17773.339844
2023-10-18  5011.899902  1960.300049  7.3099   227.815796  17732.519531

[41

In [195]:
"""
Parameters
"""

BACKTEST_DAY = 30
MODEL_TYPE = 'BL' # MVO, RP, BL, RB
TARGET_RETURN = 0.0 # target return
RISK_FREE_RATE = 0.02 # risk-free rate
REBALANCE_DAYS = 50

N = len(index_list)
index_min_weight = [0 for _ in range(N)]
index_max_weight = [1 for _ in range(N)]
WEIGHT_CONSTRAINTS = list(zip(index_min_weight, index_max_weight))

In [196]:
"""
Rebalancing
"""

def rebalance(asset_index, T):
    predicts = []
    actuals = []
    realities = []
    
    for i in range(T, len(asset_index), T):
        
        if i+T >= len(asset_index):
            break
        
        historical_data = asset_index[i-T:i]
        future_data = asset_index[i:i+T]
        
        predict, actual = tool.evaluate(historical_data, future_data, WEIGHT_CONSTRAINTS, MODEL_TYPE, TARGET_RETURN, RISK_FREE_RATE)
        predicts.append(predict)
        actuals.append(actual)
        
        reality = tool.check([1 / N for _ in range(N)], future_data, RISK_FREE_RATE)  # equally weighed
        realities.append(reality)
    
    return predicts, actuals, realities

predicts, actuals, realities = rebalance(asset_index, REBALANCE_DAYS)
print(predicts)
print(actuals)
print(realities)

[(-0.04828281102900834, 1.704331915084955, -0.04006426824765801), (-0.015407598808211416, 2.7431683610418096, -0.012907555843478816), (-0.09572248034356401, 8.813336201941304, -0.013130383057221163), (-0.01633062816650205, 3.0569408128266686, -0.011884635781648688), (-0.08409957771064322, 0.27242356572372983, -0.3821239819473352), (0.026176021229388176, 1.4061603367027056, 0.00439211736256925), (0.03995071269668515, 3.0693983794027675, 0.006499877249745304), (-0.0827721767912344, 0.24230079786700986, -0.42415121079230755), (0.021031939871139896, 1.0813172146899872, 0.0009543359313258984), (0.03763807907451551, 4.5519393704075615, 0.0038748492981215323), (-0.0004855625625545968, 0.2568576124543795, -0.07975454714698417), (-0.0037748170157557723, 2.0494682471098855, -0.011600480782896973), (0.0075155070785815495, 0.842532192263385, -0.014817823029266113), (-0.0023646395090939337, 0.6655436663049527, -0.033603564486250306), (0.0018197753626257658, 0.1416932300246954, -0.12830693911209198)

In [197]:
"""
Output
"""

def display(L1, L2, L3, normalize=False, lines_to_show=None):
    line_names = ['r1', 'v1', 's1', 'r2', 'v2', 's2', 'r3', 'v3', 's3']
    line_styles = {
        'r1': 'r-', 'v1': 'r--', 's1': 'r:',
        'r2': 'b-', 'v2': 'b--', 's2': 'b:',
        'r3': 'g-', 'v3': 'g--', 's3': 'g:'
    }

    a1, b1, c1 = zip(*L1)
    a2, b2, c2 = zip(*L2)
    a3, b3, c3 = zip(*L3)

    lines = {'r1': a1, 'v1': b1, 's1': c1, 
             'r2': a2, 'v2': b2, 's2': c2,
             'r3': a3, 'v3': b3, 's3': c3}

    def normalize_data(data):
        return (data - np.mean(data)) / np.std(data)

    if normalize:
        lines = {name: normalize_data(data) for name, data in lines.items()}

    plt.figure(figsize=(10, 6))
    for line in lines_to_show:
        plt.plot(lines[line], line_styles[line], label=line, marker='o')
    
    plt.title("R: Return\tV: Volatility\tS: Sharpe Ratio\n1: Predicted\t2: Actual\t3: Reality (Equally Weighed)\nButton 'normalize': Normalise each line")
    plt.legend()
    
    for pair in [('r1', 'r2'), ('v1', 'v2'), ('s1', 's2')]:
        if pair[0] in lines_to_show and pair[1] in lines_to_show:
            corr, _ = scipy.stats.spearmanr(lines[pair[0]], lines[pair[1]])
            print(f"Spearman correlation between {pair[0]} and {pair[1]}: {corr:.2f}")
    
    plt.show()


# @interact
interact(display, 
         L1=fixed(predicts), 
         L2=fixed(actuals),
         L3=fixed(realities), 
         normalize=True, 
         lines_to_show=SelectMultiple(options=['r1', 'r2', 'r3', 'v1', 'v2', 'v3', 's1', 's2', 's3'],
                                      value=['r1', 'r2', 'r3', 'v1', 'v2', 'v3', 's1', 's2', 's3'], 
                                      description='Lines'))



interactive(children=(Checkbox(value=True, description='normalize'), SelectMultiple(description='Lines', index…

<function __main__.display(L1, L2, L3, normalize=False, lines_to_show=None)>